# Module Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# ----
sns.set_theme(
    style="ticks",
    palette="deep",
    font_scale=1.1,
    rc={
        'figure.figsize': (12, 9),
        'figure.facecolor': 'white',
        'axes.spines.right': False,
        'axes.spines.top': False,
        'grid.alpha': 0.3,
        'axes.grid': True,
    }
    )
# ----
import warnings
warnings.filterwarnings('ignore')

# Data Reading

In [ ]:
df= pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

## Column Info
****
* **customerID** : Customer Unique ID
* **gender** : Whether the customer male or female
* **SeniorCitizen**: Whether the customer is a senior citizen or not (1, 0)
* **Partner** : Whether the customer has a partner or not (Yes, No)
* **Dependents** : Whether the customer has dependents or not (Yes, No)
* **tenure** : Number of months the customer has stayed with the company
* **PhoneService** : Whether the customer has a phone service or not (Yes, No)
* **MultipleLines** : Whether the customer has multiple lines or not (Yes, No, No phone service)
* **InternetService** : Customer’s internet service provider (DSL, Fiber optic, No)
* **OnlineSecurity** : Whether the customer has online security or not (Yes, No, No internet service)
* **OnlineBackup** : Whether the customer has online backup or not (Yes, No, No internet service)
* **DeviceProtection** : Whether the customer has device protection or not (Yes, No, No internet service)
* **TechSupport** : Whether the customer has tech support or not (Yes, No, No internet service)
* **StreamingTV** : Whether the customer has streaming TV or not (Yes, No, No internet service)
* **StreamingMovies** : Whether the customer has streaming movies or not (Yes, No, No internet service)
* **Contract** : The contract term of the customer (Month-to-month, One year, Two year)
* **PaperlessBilling** : Whether the customer has paperless billing or not (Yes, No)
* **PaymentMethod** : The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
* **MonthlyCharges** : The amount charged to the customer monthly
* **TotalCharges** : The total amount charged to the customer
* **Churn** : Whether the customer left or no  (Yes or No)
**** 
  

## Overview of Data

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

****

# Data Processing

## Data Types

In [ ]:
df.info()

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].astype('float')
df.info()

In [ ]:
df['TotalCharges'].value_counts()

****

## Missing Values

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

****

## Duplicates

In [ ]:
df.duplicated().sum()

****

## Outliers

In [ ]:
df.describe()

**No Significant Outliers**

****

# Explatory Data Analysis

In [ ]:
df.head()

In [ ]:
df.columns

****

## Churn Rate

In [ ]:
churn_rate= (df['Churn'].value_counts(normalize=True)*100).reset_index()
round(churn_rate,2)

In [ ]:
ax =sns.barplot(churn_rate, x='Churn',y='proportion',color= 'red',alpha= 0.5)
plt.title('Overall Churn Rate')
plt.text(1, 28, "26.6% Of Customers Churned", fontsize=12, color='red', 
         ha='center', va='center')

for i, bar in enumerate(ax.patches):
    if i == 1:
        bar.set_alpha(1.0)
        bar.set_linewidth(2)
plt.savefig('Overall Churn Rate.png')
plt.show()

**only 26.6% Are Churned Customers indicates an imbalanced dataset, the majority of our customer Does Not Churn**

****

### Functions For Easier Code

**Grouping For Churn Rate**

In [ ]:
def make_group(index):
    group_churn = df.groupby([index,'Churn']).agg(
    total_customer = ('customerID', 'count')).reset_index()
    group_churn['percent'] = round((group_churn['total_customer']/group_churn.groupby(index)['total_customer'].transform('sum'))*100,2)
    group_churn = group_churn[group_churn['Churn'] == 'Yes']
    # group_churn = group_churn[group_churn['Churn']=='Yes']
    return group_churn

**Bar Plot**

In [ ]:
def bar_plot(data,index,name,text,bars_names=None,bars_highlight=None):
    """data,
    index: Measured Column,
    name: name of chart,
    text: annoted text,
    bars_names : List of Bars Names,
    bars_highlight : List of bars indexes to highlight
    """
    colors = ['blue', 'brown', 'green', 'red', 'purple', 'orange', 'pink', 'gray']

    # Bar Figure
    ax =sns.barplot(data, x='Churn',y='percent',hue=index,
                alpha= 0.5)
    # Overall Churn
    
    max_value = 0
    # Bar Attribute
    for i, bar in enumerate(ax.patches):
        max_value = max(bar.get_height(), max_value)

        
        if i in bars_highlight:
            bar.set_alpha(1.0)
            bar.set_linewidth(2)
            
        if i < len(bars_names):
            
            plt.annotate(f'{bar.get_height():.1f}% of {bars_names[i]}', 
        (bar.get_x() + bar.get_width() / 2., bar.get_height()+0.5),
        ha='center', va='bottom',fontsize=12, color=colors[i], alpha= 1 )

    plt.axhline(y=26.6, xmin=0, xmax=10, alpha=0.5, linestyle='--', label= 'Overall Churn 26.6%', color = 'black')

    # Main Note
    plt.text(0, max_value*0.9, f"{text}", fontsize=14, color='black', 
             ha='center', va='center',bbox={'boxstyle': 'round', 'facecolor': 'wheat'})
    
    plt.legend(loc='lower left')
    # Title
    plt.title(f'{name} Churn Rate')
    # Export Figure
    plt.savefig(f'{name} Churn Rate().png')
    plt.show()

**Chi Square**

In [ ]:
from scipy.stats import chi2_contingency

def chi_square(index):
    contingency_table = pd.crosstab(df[index], df['Churn'])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"{index} Chi-Square Value: {chi2:.4f}, p-value: {p_value:.4f}")

## Gender Role in Churn

In [ ]:
gender_churn = make_group('gender')
gender_churn

In [ ]:
gender_chi2 = chi_square('gender')

In [ ]:
gender_bar = bar_plot(gender_churn,'gender','Gender','No Major Role By gender in Churn',['Female','Male'],[0,1])
gender_bar

**The observed differences in churn rates between genders are likely due to random chance rather than any real underlying relationship**

****

## Seniority Role in Churn.

In [ ]:
senior_churn = make_group('SeniorCitizen')

senior_map = {0:'Not Senior', 1:'Senior'}

senior_churn['SeniorCitizen'] = senior_churn['SeniorCitizen'].map(senior_map)
senior_churn

In [ ]:
df['SeniorCitizen'] = df['SeniorCitizen'].map(senior_map)

In [ ]:
senior_chi2 = chi_square('SeniorCitizen')

In [ ]:
senior_bar = bar_plot(senior_churn,'SeniorCitizen','Senior',"High Percentage of Seniors Churn Compared to non-seniors",['Non-Seniors','Seniors'],[1])

**Senior citizens have substantially higher churn rates compared to non-senior customers**

****

## Partner Effect on Churn

In [ ]:
# Calculate churn rate by partner status
partner_churn = make_group('Partner')

partner_churn

In [ ]:
partner_chi2 = chi_square('Partner')

In [ ]:
partner_bar = bar_plot(partner_churn,'Partner','Having Partner',"Customers With No Partner tend to Churn More",['No Partner','With Partner'],[0])

**Customers without partners have substantially Higher churn rates compared to those with partners**

****

## Dependents Effect on Churn

In [ ]:
# Calculate churn rate by partner status
dependent_churn = make_group('Dependents')

dependent_churn

In [ ]:
partner_chi2 = chi_square('Dependents')

In [ ]:
dependent_bar = bar_plot(dependent_churn,'Dependents','Having Dependents',"Customers With No Dependents tend to Churn More",
    ['No Dependents','With Dependents'],[0])

**Customers without dependents show markedly Higher churn behavior compared to those with dependents.**

****

## Internet Service and Churn

In [ ]:
internet_churn = make_group('InternetService')
internet_churn

In [ ]:
InternetService_chi2 = chi_square('InternetService')

In [ ]:
InternetService_churn_plot = bar_plot(internet_churn,'InternetService','Internet Service','Fiber Optic Owners Tend More To Churn',
                                      ['DSL','Fiber Optic',"No Service"],[1])

**Customers With Fiber Optic Shows Significant High Churn Rates**

****

## Additional Services

In [ ]:
df.columns

In [ ]:
services = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
           'TechSupport', 'StreamingTV', 'StreamingMovies']

In [ ]:
df_internet = df[df['InternetService'] != 'No']

In [ ]:
results = []
for service in services:
    # Create contingency table
    contingency_table = pd.crosstab(df[service], df['Churn'])
    
    # Chi-square test
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    # Calculate churn rates
    churn_rates = df.groupby(service)['Churn'].value_counts(normalize=True).unstack()
    churn_with_service = churn_rates.loc['Yes', 'Yes'] * 100
    churn_without_service = churn_rates.loc['No', 'Yes'] * 100
    
    # Calculate risk reduction
    risk_reduction = (1 - (churn_with_service / churn_without_service)) * 100
    
    results.append({
        'Service': service,
        'Chi2': chi2,
        'p_value': p_value,
        'Churn_With_Service': churn_with_service,
        'Churn_Without_Service': churn_without_service,
        'Risk_Reduction_Pct': risk_reduction
    })

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for i, service in enumerate(services):
    # Calculate churn rates for this service
    service_churn = df.groupby(service)['Churn'].apply(
        lambda x: (x == 'Yes').mean() * 100
    ).reset_index()
    
    # Create bar plot
    sns.barplot(data=service_churn, x=service, y='Churn', ax=axes[i], alpha=0.8)
    axes[i].set_title(f'{service}\n(p < 0.001)')
    axes[i].set_ylabel('Churn Rate (%)')
    axes[i].set_xlabel('')
    
    # Add value labels on bars
    for p in axes[i].patches:
        axes[i].annotate(f'{p.get_height():.1f}%', 
                        (p.get_x() + p.get_width() / 2., p.get_height()),
                        ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('Additional Services Churn.png')
plt.show()

In [ ]:
# Sort by risk reduction for better visualization
results_df = results_df.sort_values('Risk_Reduction_Pct', ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.barh(results_df['Service'], results_df['Risk_Reduction_Pct'], color='green',alpha=0.7)
plt.xlabel('Risk Reduction (%)')
plt.title('Churn Risk Reduction from Additional Services\n(How much having the service reduces churn risk)')
plt.grid(axis='x', alpha=0.3)

# Add value labels
for bar in bars:
    width = bar.get_width()
    plt.text(width + 1, bar.get_y() + bar.get_height()/2, 
             f'{width:.1f}%', ha='left', va='center')

plt.tight_layout()
plt.savefig('Additional Services Risk.png')

plt.show()

In [ ]:
# Prepare data for heatmap
heatmap_data = []
for service in services:
    churn_rates = df.groupby(service)['Churn'].apply(
        lambda x: (x == 'Yes').mean() * 100
    )
    heatmap_data.append({
        'Service': service,
        'With_Service': churn_rates.get('Yes', 0),
        'Without_Service': churn_rates.get('No', 0)
    })

heatmap_df = pd.DataFrame(heatmap_data)
heatmap_df = heatmap_df.set_index('Service')

# Create heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_df, annot=True, fmt='.1f', cmap='RdYlGn_r', 
            cbar_kws={'label': 'Churn Rate (%)'})
plt.title('Churn Rates: With vs Without Additional Services')
plt.tight_layout()
plt.savefig('Additional Services Churn Heatmap.png')
plt.show()

In [ ]:
# Create a clean summary table
summary_table = results_df[['Service', 'Churn_With_Service', 
                           'Churn_Without_Service', 'Risk_Reduction_Pct']].copy()
summary_table.columns = ['Service', 'Churn Rate With Service (%)', 
                        'Churn Rate Without Service (%)', 'Risk Reduction (%)']
summary_table = summary_table.round(1)

print("Summary of Additional Services Impact on Churn:")
print(summary_table.to_string(index=False))

****

## Phone Service vs Multiple Lines impact

In [ ]:
df.columns

In [ ]:
df[['PhoneService','MultipleLines']].value_counts()

In [ ]:
df['PhoneServiceType'] = df['PhoneService'].map({'Yes': 'Phone Only', 'No': 'No Phone'})
df.loc[(df['PhoneService'] == 'Yes') & (df['MultipleLines'] == 'Yes'), 'PhoneServiceType'] = 'Multiple Lines'

# Verify the new categories
print("Phone Service Type Distribution:")
print(df['PhoneServiceType'].value_counts())

In [ ]:
chi_square('PhoneServiceType')

In [ ]:
# Calculate churn rates and confidence intervals
import math
from scipy.stats import norm

def proportion_confidence_interval(successes, trials, confidence=0.95):
    """Calculate confidence interval for a proportion"""
    p = successes / trials
    z = norm.ppf(1 - (1 - confidence) / 2)
    margin_error = z * math.sqrt((p * (1 - p)) / trials)
    return p * 100, (p - margin_error) * 100, (p + margin_error) * 100

In [ ]:
phone_types = ['No Phone', 'Phone Only', 'Multiple Lines']
churn_data = []

for phone_type in phone_types:
    subset = df[df['PhoneServiceType'] == phone_type]
    total = len(subset)
    churn_count = (subset['Churn'] == 'Yes').sum()
    
    churn_rate, ci_lower, ci_upper = proportion_confidence_interval(churn_count, total)
    
    churn_data.append({
        'PhoneServiceType': phone_type,
        'Total_Customers': total,
        'Churn_Count': churn_count,
        'Churn_Rate': churn_rate,
        'CI_Lower': ci_lower,
        'CI_Upper': ci_upper
    })

churn_df = pd.DataFrame(churn_data)
print("\nChurn Rates by Phone Service Type:")
print(churn_df)

In [ ]:
plt.figure(figsize=(12, 6))

ax = sns.pointplot(data=churn_df, x='PhoneServiceType', y='Churn_Rate', 
                   join=False, capsize=0.1, color='red', markers='D', scale=1.2)

plt.title('Churn Rates with 95% Confidence Intervals by Phone Service Type', 
          fontsize=14, fontweight='bold')
plt.xlabel('Phone Service Type', fontweight='bold')
plt.ylabel('Churn Rate (%)', fontweight='bold')

for i, row in churn_df.iterrows():
    plt.text(i+0.12, row['Churn_Rate']-0.05 , f'{row["Churn_Rate"]:.1f}%', 
             ha='center', va='bottom', fontweight='bold', fontsize=11)

# Add horizontal reference line for overall churn rate
overall_churn = (df['Churn'] == 'Yes').mean() * 100
plt.axhline(y=overall_churn, color='gray', linestyle='--', alpha=0.7, label=f'Overall Churn: {overall_churn:.1f}%')
plt.legend()

plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('Churn Rates with 95% Confidence Intervals by Phone Service Type')
plt.show()

****

## How does Contract type impact churn?

In [ ]:
df['Contract'].value_counts()

In [ ]:
contract_churn = make_group('Contract')
contract_churn['retention'] = 100 - contract_churn['percent'] 
contract_churn

In [ ]:
contact_chi2 = chi_square('Contract')

In [ ]:
contract_bar= bar_plot(contract_churn,'Contract','Contract',"Month-to-Month Customers Show High Churn Perentage",['Month-to-Month','One Year','Two Year'],[0])

****

## Paperless Billing impact


In [ ]:
paperless_grp = make_group('PaperlessBilling')
paperless_grp

In [ ]:
paperless_chi2 = chi_square('PaperlessBilling')

In [ ]:
# Create contingency table
contingency_table = pd.crosstab(df['PaperlessBilling'], df['Churn'])
print("Contingency Table:")
print(contingency_table)

In [ ]:
# Calculate Odds Ratio
a = contingency_table.loc['Yes', 'Yes']  # Paperless & Churned
b = contingency_table.loc['Yes', 'No']   # Paperless & Not Churned
c = contingency_table.loc['No', 'Yes']   # Non-Paperless & Churned
d = contingency_table.loc['No', 'No']    # Non-Paperless & Not Churned

odds_ratio = (a * d) / (b * c)
print(f"Odds Ratio: {odds_ratio:.4f}")

# Interpretation
if odds_ratio > 1:
    direction = "higher"
else:
    direction = "lower"
print(f"Paperless billing customers have {direction} odds of churning")

In [ ]:
paperless_bar = bar_plot(paperless_grp,'PaperlessBilling','Paperless','Paperless billing customers have higher odds of churning',['Papered Billing','Paperless Billing'],[1])

****

## Payment Method with the highest churn

In [ ]:
df['PaymentMethod'].value_counts()

In [ ]:
payment_churn = make_group('PaymentMethod')
payment_churn

In [ ]:
bar_plot(payment_churn, 'PaymentMethod','Payment Method','Customers Paying With Electronic Checks Have High Percentage of Churn',
         ['Bank Transfer','Credit Card','Electronic Cards','Mailed Check'],[2])

****

## MonthlyCharges and TotalCharges for churned vs retained

In [ ]:
# Ensure TotalCharges is numeric (common issue in telco datasets)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Check for missing values
print("Missing values in TotalCharges:", df['TotalCharges'].isna().sum())

# Handle missing values if any
df_clean = df.dropna(subset=['TotalCharges']).copy()

In [ ]:
# Group by churn status and calculate statistics
stats_summary = df_clean.groupby('Churn')[['MonthlyCharges', 'TotalCharges']].agg([
    'mean', 'median', 'std', 'min', 'max'
]).round(2)

print("Descriptive Statistics by Churn Status:")
stats_summary


In [ ]:
# Calculate percentage differences
mean_monthly_churned = df_clean[df_clean['Churn']=='Yes']['MonthlyCharges'].mean()
mean_monthly_retained = df_clean[df_clean['Churn']=='No']['MonthlyCharges'].mean()
pct_diff_monthly = ((mean_monthly_churned - mean_monthly_retained) / mean_monthly_retained) * 100

mean_total_churned = df_clean[df_clean['Churn']=='Yes']['TotalCharges'].mean()
mean_total_retained = df_clean[df_clean['Churn']=='No']['TotalCharges'].mean()
pct_diff_total = ((mean_total_churned - mean_total_retained) / mean_total_retained) * 100

print(f"\nPercentage Difference in Means:")
print(f"MonthlyCharges: {pct_diff_monthly:.1f}%")
print(f"TotalCharges: {pct_diff_total:.1f}%")

In [ ]:
from scipy.stats import mannwhitneyu

# Split data into churned and retained groups
churned = df_clean[df_clean['Churn'] == 'Yes']
retained = df_clean[df_clean['Churn'] == 'No']

# Mann-Whitney U test for MonthlyCharges
u_monthly, p_monthly = mannwhitneyu(
    churned['MonthlyCharges'], 
    retained['MonthlyCharges'],
    alternative='two-sided'
)

# Mann-Whitney U test for TotalCharges
u_total, p_total = mannwhitneyu(
    churned['TotalCharges'], 
    retained['TotalCharges'], 
    alternative='two-sided'
)

print("Mann-Whitney U Test Results:")
print(f"MonthlyCharges: U = {u_monthly:.0f}, p = {p_monthly:.6f}")
print(f"TotalCharges: U = {u_total:.0f}, p = {p_total:.6f}")

In [ ]:
def cohens_d(group1, group2):
    """Calculate Cohen's d for effect size"""
    from math import sqrt
    
    # Mean difference
    mean_diff = group1.mean() - group2.mean()
    
    # Pooled standard deviation
    n1, n2 = len(group1), len(group2)
    var1, var2 = group1.var(), group2.var()
    pooled_std = sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1 + n2 - 2))
    
    return mean_diff / pooled_std

# Calculate Cohen's d for both variables
d_monthly = cohens_d(churned['MonthlyCharges'], retained['MonthlyCharges'])
d_total = cohens_d(churned['TotalCharges'], retained['TotalCharges'])

print(f"\nCohen's d Effect Size:")
print(f"MonthlyCharges: d = {d_monthly:.3f}")
print(f"TotalCharges: d = {d_total:.3f}")

# Interpret Cohen's d
def interpret_cohens_d(d):
    if abs(d) < 0.2: return "negligible"
    elif abs(d) < 0.5: return "small"
    elif abs(d) < 0.8: return "medium"
    else: return "large"

print(f"MonthlyCharges effect: {interpret_cohens_d(d_monthly)}")
print(f"TotalCharges effect: {interpret_cohens_d(d_total)}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# MonthlyCharges box plot
sns.boxplot(data=df_clean, x='Churn', y='MonthlyCharges', ax=ax1, palette=['lightgreen', 'lightcoral'])
ax1.set_title('Monthly Charges by Churn Status\n(Mann-Whitney U: p < 0.001)')
ax1.set_ylabel('Monthly Charges ($)')
ax1.set_xlabel('Churn Status')

# Add median values to box plot
medians_monthly = df_clean.groupby('Churn')['MonthlyCharges'].median()
for i, median in enumerate(medians_monthly):
    ax1.text(i, median, f'Median: ${median:.0f}', 
             ha='center', va='bottom', color='black')

# TotalCharges box plot
sns.boxplot(data=df_clean, x='Churn', y='TotalCharges', ax=ax2, palette=['lightgreen', 'lightcoral'])
ax2.set_title('Total Charges by Churn Status\n(Mann-Whitney U: p < 0.001)')
ax2.set_ylabel('Total Charges ($)')
ax2.set_xlabel('Churn Status')

# Add median values to box plot
medians_total = df_clean.groupby('Churn')['TotalCharges'].median()
for i, median in enumerate(medians_total):
    ax2.text(i, median, f'Median: ${median:.0f}', 
             ha='center', va='bottom', color='black')

plt.tight_layout()
plt.savefig('Charges by Churn Status boxplot.png')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# MonthlyCharges violin plot
sns.violinplot(data=df_clean, x='Churn', y='MonthlyCharges', ax=ax1, palette=['lightgreen', 'lightcoral'])
ax1.set_title('Monthly Charges Distribution by Churn Status\n(Violin Plot)')
ax1.set_ylabel('Monthly Charges ($)')
ax1.set_xlabel('Churn Status')
ax1.text(0.5, 50, f'High Churn Rates Accure at High Monthly Charges', 
             ha='center', va='bottom', color='black',
bbox={'boxstyle': 'round', 'facecolor': 'wheat'},alpha=0.8)


# TotalCharges violin plot
sns.violinplot(data=df_clean, x='Churn', y='TotalCharges', ax=ax2, palette=['lightgreen', 'lightcoral'])
ax2.set_title('Total Charges Distribution by Churn Status\n(Violin Plot)')
ax2.set_ylabel('Total Charges ($)')
ax2.set_xlabel('Churn Status')

plt.tight_layout()
plt.savefig('Charges by Churn Status violin plot.png')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# MonthlyCharges KDE plot
sns.kdeplot(data=churned, x='MonthlyCharges', label='Churned', fill=True, alpha=0.5, ax=ax1)
sns.kdeplot(data=retained, x='MonthlyCharges', label='Retained', fill=True, alpha=0.5, ax=ax1)
ax1.set_title('Monthly Charges Distribution\n(KDE Plot)')
ax1.set_xlabel('Monthly Charges ($)')
ax1.set_ylabel('Density')
ax1.legend()
ax1.grid(True, alpha=0.3)

# TotalCharges KDE plot
sns.kdeplot(data=churned, x='TotalCharges', label='Churned', fill=True, alpha=0.5, ax=ax2)
sns.kdeplot(data=retained, x='TotalCharges', label='Retained', fill=True, alpha=0.5, ax=ax2)
ax2.set_title('Total Charges Distribution\n(KDE Plot)')
ax2.set_xlabel('Total Charges ($)')
ax2.set_ylabel('Density')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Charges by Churn Status Denisty plot.png')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# MonthlyCharges KDE plot
sns.histplot(data=churned, x='MonthlyCharges', label='Churned',stat='percent', fill=True,bins=10, alpha=0.5, ax=ax1)
sns.histplot(data=retained, x='MonthlyCharges', label='Retained',stat='percent', fill=True,bins=10, alpha=0.5, ax=ax1)

ax1.set_title('Monthly Charges Distribution PCT%')
ax1.set_xlabel('Monthly Charges ($)')
ax1.set_ylabel('PCT %')
ax1.legend()
ax1.grid(True, alpha=0.3)

# TotalCharges KDE plot
sns.histplot(data=churned, x='TotalCharges', label='Churned',stat='percent', fill=True, alpha=0.5, bins=10,ax=ax2)

sns.histplot(data=retained, x='TotalCharges', label='Retained',stat='percent', fill=True, alpha=0.5,bins=10, ax=ax2)

ax2.set_title('Total Charges Distribution PCT%')
ax2.set_xlabel('Total Charges ($)')
ax2.set_ylabel('PCT %')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Churn By Charges Denisty plot.png')

plt.show()

****

## Customer Tenure & Loyalty

### Average tenure of churned vs loyal customers

In [ ]:
# Group by churn status and calculate statistics
tenure_churn = df.groupby('Churn')['tenure'].agg([
    'mean', 'median', 'min', 'max'
]).round(2)
tenure_churn


In [ ]:
churned_tenure = df[df['Churn']=='Yes']['tenure']
retained_tenure = df[df['Churn']=='No']['tenure']
u_tenure, p_tenure = mannwhitneyu(churned_tenure, retained_tenure, alternative='two-sided')
print(f"tenure Mann-Whitney U test p-value: {p_tenure:.4f}, u-value: {u_tenure}")

In [ ]:
sns.kdeplot(data=churned, x='tenure', label='Churned', fill=True, alpha=0.5)
sns.kdeplot(data=retained, x='tenure', label='Retained', fill=True, alpha=0.5)
plt.title('Tenure Distribution\n(KDE Plot)')
plt.xlabel('Tenure (M)')
plt.ylabel('Density')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('Tenure by Churn Status Denisty plot.png')

plt.show()

In [ ]:
# Calculate percentiles for each group

percentiles = [25, 50, 75, 90, 95]
churned_percentiles = np.percentile(churned_tenure, percentiles)
retained_percentiles = np.percentile(retained_tenure, percentiles)

percentile_df = pd.DataFrame({
    'Percentile': percentiles,
    'Churned': churned_percentiles,
    'Retained': retained_percentiles
})

print("\nTenure Percentiles by Churn Status (months):")
print(percentile_df)

# Calculate the difference at key percentiles
print(f"\nKey Differences:")
print(f"Median tenure: Churned {churned_percentiles[1]:.1f} months vs Retained {retained_percentiles[1]:.1f} months")
print(f"75th percentile: Churned {churned_percentiles[2]:.1f} months vs Retained {retained_percentiles[2]:.1f} months")

In [ ]:
plt.figure(figsize=(12, 6))

# Create histogram bins

# Plot histograms
sns.histplot(retained_tenure,  alpha=0.7, label='Retained',color='green')
sns.histplot(churned_tenure,  alpha=0.6, label='Churned',color='red')

plt.xlabel('Tenure (months)')
plt.ylabel('Number of Customers')
plt.title(f'Customer Tenure Distribution by Churn Status')
plt.legend()
plt.grid(True, alpha=0.3)

# Add vertical lines for medians
plt.axvline(retained_tenure.median(), color='darkgreen', linestyle='--', alpha=0.8, 
            label=f'Retained Median: {retained_tenure.median():.0f} mo')
plt.axvline(churned_tenure.median(), color='darkred', linestyle='--', alpha=0.8,
            label=f'Churned Median: {churned_tenure.median():.0f} mo')
plt.axvspan(0, 12, alpha=0.2, color='orange', label='High Risk Period (0-12 mo)')

plt.legend()

plt.tight_layout()
plt.savefig('Customer Tenure Distribution by Churn Status histplot.png')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

# Function to create ECDF
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n + 1) / n
    return x, y

# Calculate ECDF for both groups
x_churned, y_churned = ecdf(churned_tenure)
x_retained, y_retained = ecdf(retained_tenure)

# Plot ECDF
plt.plot(x_retained, y_retained, label='Retained', color='green', linewidth=2)
plt.plot(x_churned, y_churned, label='Churned', color='red', linewidth=2)

plt.xlabel('Tenure (months)')
plt.ylabel('Cumulative Proportion')
plt.title('Empirical Cumulative Distribution Function (ECDF) of Customer Tenure')
plt.legend()
plt.grid(True, alpha=0.3)

# Add key reference points
critical_points = [6, 12, 24]
for point in critical_points:
    # Find the proportion churned by this point
    prop_churned = (churned_tenure <= point).mean()
    prop_retained = (retained_tenure <= point).mean()
    
    plt.axvline(point, color='gray', linestyle=':', alpha=0.7)
    plt.text(point, 0.5, f'{point} mo\n{prop_churned*100:.0f}% churned\n{prop_retained*100:.0f}% retained', 
             ha='center', va='center', fontsize=8, 
             bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))

plt.tight_layout()
plt.savefig('(ECDF) of Customer Tenure')
plt.show()

In [ ]:
# Create survival-style plot (1 - ECDF)
plt.figure(figsize=(12, 6))

# Calculate survival curves (proportion remaining at each tenure)
max_tenure = max(df['tenure'].max(), 72)  # Cap at 72 months for clarity
tenure_range = np.arange(0, max_tenure + 1)

survival_churned = [(churned_tenure > t).mean() for t in tenure_range]
survival_retained = [(retained_tenure > t).mean() for t in tenure_range]

plt.plot(tenure_range, survival_retained, label='Retained Customers', color='green', linewidth=3)
plt.plot(tenure_range, survival_churned, label='Churned Customers', color='red', linewidth=3)

plt.xlabel('Tenure (months)')
plt.ylabel('Proportion Still Active')
plt.title('Customer Retention Over Time\n(Survival-style Analysis)')
plt.legend()
plt.grid(True, alpha=0.3)

# Add key milestones
for months in [12, 24, 36]:
    plt.axvline(months, color='gray', linestyle=':', alpha=0.5)
    plt.text(months, 0.1, f'{months} mo', ha='center', va='bottom', rotation=90)

plt.tight_layout()
plt.savefig('Customer Retention Over Time Survival-style Analysis.png')
plt.show()

In [ ]:
# Create detailed summary table
summary_data = {
    'Metric': [
        'Sample Size',
        'Mean Tenure (months)',
        'Median Tenure (months)',
        'Standard Deviation',
        '25th Percentile',
        '75th Percentile',
        'Proportion < 6 months',
        'Proportion < 12 months',
        'Proportion > 24 months'
    ],
    'Churned': [
        len(churned_tenure),
        churned_tenure.mean(),
        churned_tenure.median(),
        churned_tenure.std(),
        np.percentile(churned_tenure, 25),
        np.percentile(churned_tenure, 75),
        (churned_tenure < 6).mean() * 100,
        (churned_tenure < 12).mean() * 100,
        (churned_tenure > 24).mean() * 100
    ],
    'Retained': [
        len(retained_tenure),
        retained_tenure.mean(),
        retained_tenure.median(),
        retained_tenure.std(),
        np.percentile(retained_tenure, 25),
        np.percentile(retained_tenure, 75),
        (retained_tenure < 6).mean() * 100,
        (retained_tenure < 12).mean() * 100,
        (retained_tenure > 24).mean() * 100
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df['Churned'] = summary_df['Churned'].round(2)
summary_df['Retained'] = summary_df['Retained'].round(2)

print("\n" + "="*70)
print("COMPREHENSIVE TENURE ANALYSIS SUMMARY")
print("="*70)
print(summary_df.to_string(index=False))

# Calculate key business insights
critical_period = 12  # First year
churned_in_first_year = (churned_tenure <= critical_period).mean() * 100
retained_past_first_year = (retained_tenure > critical_period).mean() * 100

print(f"\nKEY BUSINESS INSIGHTS:")
print(f"• {churned_in_first_year:.1f}% of churned customers left within first {critical_period} months")
print(f"• {retained_past_first_year:.1f}% of retained customers stayed beyond {critical_period} months")
print(f"• Median tenure difference: {retained_tenure.median() - churned_tenure.median():.1f} months")

### Critical churn periods

In [ ]:
# Create tenure groups
df['TenureGroup'] = pd.cut(df['tenure'],
bins=[0, 6, 12, 24, 36, 72],
labels=['0-6', '7-12', '13-24', '25-36', '37+'])

In [ ]:
# Calculate churn rate by tenure group
tenure_churn = df.groupby('TenureGroup')['Churn'].value_counts(normalize=True).unstack()*100
tenure_churn

In [ ]:
tenure_churn2 = make_group('TenureGroup')
tenure_churn2['pct_of_churn'] = round((tenure_churn2['total_customer']/tenure_churn2['total_customer'].sum())*100,2)
tenure_churn2

In [ ]:
ax = sns.barplot(tenure_churn2, x='TenureGroup',y='pct_of_churn',color='red',alpha=0.7)

for i, bar in enumerate(ax.patches):    
        plt.annotate(f'{bar.get_height():.1f}%', 
        (bar.get_x() + bar.get_width() / 2., bar.get_height()+0.5),
        ha='center', va='bottom',fontsize=12, alpha= 0.7, color= 'red')
    
        if i ==0:
            bar.set_alpha(1.0)
            bar.set_linewidth(2)
            
            


plt.title('Churn Rate by Tenure Group')
plt.ylabel('Churn Rate (%)')
plt.xlabel('Tenure Group (months)')
plt.xticks(rotation=45)
plt.savefig('PCT of Churn by Tenure Group.png')
plt.show()

In [ ]:
tenure_churn['Yes'].plot(kind='bar', color='red')
plt.title('Churn Rate by Tenure Group')
plt.ylabel('Churn Rate (%)')
plt.xlabel('Tenure Group (months)')
plt.xticks(rotation=45)
plt.savefig('Churn Rate by Tenure Group.png')
plt.show()

In [ ]:
# Calculate raw churn rate by tenure
tenure_churn = df.groupby('tenure')['Churn'].apply(
    lambda x: (x == 'Yes').mean() * 100
).reset_index()
tenure_churn.columns = ['Tenure', 'ChurnRate']

# Add rolling average to smooth the line
tenure_churn['RollingAvg'] = tenure_churn['ChurnRate'].rolling(window=3, center=True).mean()

plt.figure(figsize=(14, 7))

# Plot both raw data and smoothed line
plt.plot(tenure_churn['Tenure'], tenure_churn['ChurnRate'], 
         alpha=0.4, color='gray', linewidth=1, label='Raw Data')
plt.plot(tenure_churn['Tenure'], tenure_churn['RollingAvg'], 
         linewidth=3, color='red', label='3-Month Moving Average')

plt.xlabel('Customer Tenure (Months)')
plt.ylabel('Churn Rate (%)')
plt.title('Churn Rate vs Customer Tenure\n(Moving Average)')
plt.grid(True, alpha=0.3)
plt.legend()

# Highlight critical periods
critical_periods = [(0, 3, 'Onboarding Risk'), (10, 13, '1-Year Review'), 
                    (22, 25, '2-Year Review')]

for start, end, label in critical_periods:
    plt.axvspan(start, end, alpha=0.2, color='orange')
    plt.text((start + end) / 2, tenure_churn['ChurnRate'].max() * 0.9, 
             label, ha='center', va='center')

plt.tight_layout()
plt.savefig("Churn Rate vs Customer Tenure")

plt.show()

****

## The Cost of Churn

In [ ]:
df.columns

In [ ]:
def calculate_churn_costs(df):
    """Calculate comprehensive churn costs"""
    
    # Identify churned customers
    churned_customers = df[df['Churn'] == 'Yes']
    n_churned = len(churned_customers)
    
    # 1. Lost Monthly Revenue
    monthly_revenue_lost = churned_customers['MonthlyCharges'].sum()
    
    # 2. Customer Acquisition Cost Lost
    cac = 300  # Use your actual CAC or industry average
    acquisition_cost_lost = n_churned * cac
    
    # 3. Lifetime Value Lost
    avg_monthly_charge = churned_customers['MonthlyCharges'].mean()
    monthly_churn_rate = n_churned / len(df)
    gross_margin = 0.8  # 80% margin
    
    avg_ltv = (avg_monthly_charge * gross_margin) / monthly_churn_rate
    total_ltv_lost = avg_ltv * n_churned
    
    # 4. Total Cost of Churn (immediate + long-term)
    total_immediate_cost = monthly_revenue_lost + acquisition_cost_lost
    total_cost = total_immediate_cost + total_ltv_lost
    
    return {
        'monthly_revenue_lost': monthly_revenue_lost,
        'acquisition_cost_lost': acquisition_cost_lost,
        'avg_ltv_per_customer': avg_ltv,
        'total_ltv_lost': total_ltv_lost,
        'total_immediate_cost': total_immediate_cost,
        'total_churn_cost': total_cost,
        'customers_churned': n_churned
    }

# Calculate all costs
costs = calculate_churn_costs(df)

print("CHURN COST ANALYSIS")
print("=" * 50)
print(f"Customers Churned: {costs['customers_churned']:,}")
print(f"1. Lost Monthly Revenue: ${costs['monthly_revenue_lost']:,.2f}")
print(f"2. Acquisition Cost Wasted: ${costs['acquisition_cost_lost']:,.2f}")
print(f"3. Lifetime Value Lost: ${costs['total_ltv_lost']:,.2f}")
print("-" * 50)
print(f"TOTAL IMMEDIATE COST: ${costs['total_immediate_cost']:,.2f}")
print(f"TOTAL LONG-TERM COST: ${costs['total_churn_cost']:,.2f}")

****

## Top factors driving churn

In [ ]:
from scipy.stats import chi2_contingency, pointbiserialr

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve, 
                             confusion_matrix, classification_report)
from sklearn.feature_selection import mutual_info_classif
from sklearn.calibration import calibration_curve

import xgboost as xgb

import shap


In [ ]:
df_analysis = df.copy()

# Convert target to binary
df_analysis['Churn_binary'] = (df_analysis['Churn'] == 'Yes').astype(int)

# Select features for analysis (excluding ID and target)
feature_columns = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
                   'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
                   'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
                   'MonthlyCharges', 'TotalCharges']

# Prepare data for different methods
X = df_analysis[feature_columns]
y = df_analysis['Churn_binary']

# Create encoded version for Random Forest and Mutual Information
X_encoded = X.copy()
label_encoders = {}
for col in X_encoded.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le

In [ ]:
def cramers_v(x, y):
    """Calculate Cramér's V statistic for categorical-categorical association"""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Calculate Cramér's V for all categorical variables
categorical_features = X.select_dtypes(include=['object']).columns
cramers_results = []

for feature in categorical_features:
    try:
        v_stat = cramers_v(X[feature], y)
        cramers_results.append({
            'Feature': feature,
            'Cramers_V': v_stat,
            'Method': 'Cramers_V'
        })
    except:
        continue

cramers_df = pd.DataFrame(cramers_results)
print("Cramér's V Results (Top 10):")
print(cramers_df.sort_values('Cramers_V', ascending=False).head(10))

In [ ]:
# Calculate Point-Biserial correlation for numerical features
numerical_features = X.select_dtypes(include=[np.number]).columns
pointbiserial_results = []

for feature in numerical_features:
    try:
        corr, p_value = pointbiserialr(X[feature], y)
        pointbiserial_results.append({
            'Feature': feature,
            'PointBiserial_Corr': abs(corr),  # Use absolute value for ranking
            'P_Value': p_value,
            'Method': 'PointBiserial'
        })
    except:
        continue

pointbiserial_df = pd.DataFrame(pointbiserial_results)
print("\nPoint-Biserial Correlation Results:")
print(pointbiserial_df.sort_values('PointBiserial_Corr', ascending=False))

In [ ]:
# Train Random Forest and get feature importance
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_encoded, y)

# Get feature importance
rf_importance = pd.DataFrame({
    'Feature': feature_columns,
    'RF_Importance': rf_model.feature_importances_,
    'Method': 'RandomForest'
})

print("\nRandom Forest Feature Importance (Top 10):")
print(rf_importance.sort_values('RF_Importance', ascending=False).head(10))

In [ ]:
# Calculate Mutual Information
mi_scores = mutual_info_classif(X_encoded, y, random_state=42)
mi_results = pd.DataFrame({
    'Feature': feature_columns,
    'Mutual_Info': mi_scores,
    'Method': 'MutualInfo'
})

print("\nMutual Information Scores (Top 10):")
print(mi_results.sort_values('Mutual_Info', ascending=False).head(10))

In [ ]:
# Combine all results into one dataframe
combined_results = []

# Helper function to normalize scores
def normalize_scores(series):
    return (series - series.min()) / (series.max() - series.min())

# Add Cramér's V results
for _, row in cramers_df.iterrows():
    combined_results.append({
        'Feature': row['Feature'],
        'Cramers_V': row['Cramers_V'],
        'PointBiserial_Corr': 0,
        'RF_Importance': 0,
        'Mutual_Info': 0
    })

# Add Point-Biserial results
for _, row in pointbiserial_df.iterrows():
    # Check if feature already exists
    existing = [r for r in combined_results if r['Feature'] == row['Feature']]
    if existing:
        existing[0]['PointBiserial_Corr'] = row['PointBiserial_Corr']
    else:
        combined_results.append({
            'Feature': row['Feature'],
            'Cramers_V': 0,
            'PointBiserial_Corr': row['PointBiserial_Corr'],
            'RF_Importance': 0,
            'Mutual_Info': 0
        })

# Add Random Forest results
for _, row in rf_importance.iterrows():
    existing = [r for r in combined_results if r['Feature'] == row['Feature']]
    if existing:
        existing[0]['RF_Importance'] = row['RF_Importance']
    else:
        combined_results.append({
            'Feature': row['Feature'],
            'Cramers_V': 0,
            'PointBiserial_Corr': 0,
            'RF_Importance': row['RF_Importance'],
            'Mutual_Info': 0
        })

# Add Mutual Information results
for _, row in mi_results.iterrows():
    existing = [r for r in combined_results if r['Feature'] == row['Feature']]
    if existing:
        existing[0]['Mutual_Info'] = row['Mutual_Info']
    else:
        combined_results.append({
            'Feature': row['Feature'],
            'Cramers_V': 0,
            'PointBiserial_Corr': 0,
            'RF_Importance': 0,
            'Mutual_Info': row['Mutual_Info']
        })

# Create final combined dataframe
final_df = pd.DataFrame(combined_results)

# Calculate normalized scores and overall rank
final_df['Cramers_V_Norm'] = normalize_scores(final_df['Cramers_V'])
final_df['PointBiserial_Norm'] = normalize_scores(final_df['PointBiserial_Corr'])
final_df['RF_Norm'] = normalize_scores(final_df['RF_Importance'])
final_df['MI_Norm'] = normalize_scores(final_df['Mutual_Info'])

# Calculate overall score (average of normalized scores)
final_df['Overall_Score'] = (final_df['Cramers_V_Norm'] + 
                            final_df['PointBiserial_Norm'] + 
                            final_df['RF_Norm'] + 
                            final_df['MI_Norm']) / 4

# Sort by overall score
final_ranked = final_df.sort_values('Overall_Score', ascending=False)

print("\n" + "="*80)
print("FINAL RANKING: Top Factors Driving Churn")
print("="*80)
print(final_ranked[['Feature', 'Overall_Score', 'Cramers_V', 'PointBiserial_Corr', 
                   'RF_Importance', 'Mutual_Info']].round(4).head(10))

In [ ]:
# Prepare data for visualization
top_features = final_ranked.head(10).copy()
top_features = top_features.sort_values('Overall_Score', ascending=False)

plt.figure(figsize=(12, 8))


sns.barplot(data=top_features, x='Overall_Score', y='Feature', alpha=0.8, color='blue',palette="rocket")


plt.xlabel('Normalized Importance Score')
plt.title('Top 10 Churn Drivers by Multiple Feature Importance Methods')
plt.legend()
plt.grid(axis='x', alpha=0.3)


plt.tight_layout()
plt.savefig('Top 10 Churn Drivers by Multiple Feature Importance Methods.png')
plt.show()

****

## High-risk customer profile

In [ ]:
# Define high-risk rules based on decision tree and business knowledge
def create_customer_segments(df):
    """Create customer segments based on high-risk characteristics"""
    
    segments = []
    
    # Rule 1: Month-to-month + High Monthly Charges + No Support
    condition1 = (df['Contract'] == 'Month-to-month') & \
                 (df['MonthlyCharges'] > df['MonthlyCharges'].median()) & \
                 (df['TechSupport'] == 'No') & \
                 (df['OnlineSecurity'] == 'No')
    segments.append(('Premium_NoSupport', condition1))
    
    # Rule 2: Month-to-month + Short Tenure + Electronic Check
    condition2 = (df['Contract'] == 'Month-to-month') & \
                 (df['tenure'] <= 12) & \
                 (df['PaymentMethod'] == 'Electronic check')
    segments.append(('New_Echeck', condition2))
    
    # Rule 3: Fiber Optic + No Online Security
    condition3 = (df['InternetService'] == 'Fiber optic') & \
                 (df['OnlineSecurity'] == 'No')
    segments.append(('Fiber_NoSecurity', condition3))
    
    # Rule 4: Senior + No Partner + No Dependents
    condition4 = (df['SeniorCitizen'] == 1) & \
                 (df['Partner'] == 'No') & \
                 (df['Dependents'] == 'No')
    segments.append(('Senior_Alone', condition4))
    
    # Rule 5: High Monthly + Multiple Services but No Contract
    condition5 = (df['MonthlyCharges'] > 75) & \
                 (df['Contract'] == 'Month-to-month') & \
                 ((df['StreamingTV'] == 'Yes') | (df['StreamingMovies'] == 'Yes'))
    segments.append(('HighSpender_NoCommitment', condition5))
    
    return segments

# Apply segmentation rules
segments = create_customer_segments(df)
segment_results = []

for segment_name, condition in segments:
    segment_data = df[condition]
    segment_size = len(segment_data)
    
    if segment_size > 0:
        churn_rate = (segment_data['Churn'] == 'Yes').mean() * 100
        overall_churn = (df['Churn'] == 'Yes').mean() * 100
        risk_multiplier = churn_rate / overall_churn
        
        segment_results.append({
            'Segment': segment_name,
            'Size': segment_size,
            'Churn_Rate': churn_rate,
            'Overall_Churn': overall_churn,
            'Risk_Multiplier': risk_multiplier,
            'Condition': condition.sum()  # Count of customers in segment
        })

segment_df = pd.DataFrame(segment_results)
segment_df = segment_df.sort_values('Churn_Rate', ascending=False)

print("High-Risk Customer Segments:")
print(segment_df.round(2))

****

****

# Predictive modeling

In [ ]:
df.columns

In [ ]:
# Prepare the data
df_model = df.copy()

# Convert target to binary
df_model['Churn'] = (df_model['Churn'] == 'Yes').astype(int)

# Handle categorical variables
categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 
                   'MultipleLines', 'InternetService', 'OnlineSecurity', 
                   'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                   'StreamingTV', 'StreamingMovies', 'Contract', 
                   'PaperlessBilling', 'PaymentMethod','TenureGroup','PhoneServiceType', 'SeniorCitizen']

# One-hot encoding for categorical variables
df_encoded = pd.get_dummies(df_model, columns=categorical_cols, drop_first=True)

# Define features and target
X = df_encoded.drop(['customerID', 'Churn'], axis=1)
y = df_encoded['Churn']

# Handle missing values if any
X = X.fillna(X.median())

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Churn rate in training: {y_train.mean():.3f}")
print(f"Churn rate in test: {y_test.mean():.3f}")

In [ ]:
# Initialize models
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=50,
    random_state=42,
    class_weight='balanced'  # Handle class imbalance
)

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1])  # Handle imbalance
)

# Train models
print("Training Random Forest...")
rf_model.fit(X_train, y_train)

print("Training XGBoost...")
xgb_model.fit(X_train, y_train)

# Cross-validation scores
rf_cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='roc_auc')
xgb_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='roc_auc')

print(f"Random Forest CV AUC: {rf_cv_scores.mean():.3f} (+/- {rf_cv_scores.std() * 2:.3f})")
print(f"XGBoost CV AUC: {xgb_cv_scores.mean():.3f} (+/- {xgb_cv_scores.std() * 2:.3f})")

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    """Comprehensive model evaluation"""
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred_proba)
    
    # Print results
    print(f"\n{model_name} Performance:")
    print("="*50)
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"AUC-ROC:   {auc_roc:.4f}")
    
    # Classification report
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=['Not Churn', 'Churn']))
    
    return {
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'metrics': {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc_roc': auc_roc
        }
    }

# Evaluate both models
rf_results = evaluate_model(rf_model, X_test, y_test, "Random Forest")
xgb_results = evaluate_model(xgb_model, X_test, y_test, "XGBoost")

# Compare models
comparison_df = pd.DataFrame({
    'Random Forest': rf_results['metrics'],
    'XGBoost': xgb_results['metrics']
})
print("\nModel Comparison:")
print(comparison_df.round(4))

In [ ]:
# Calculate ROC curves
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_results['y_pred_proba'])
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, xgb_results['y_pred_proba'])

# Calculate AUC scores
auc_rf = roc_auc_score(y_test, rf_results['y_pred_proba'])
auc_xgb = roc_auc_score(y_test, xgb_results['y_pred_proba'])

plt.figure(figsize=(10, 8))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.3f})', linewidth=2)
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {auc_xgb:.3f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Churn Prediction Models')
plt.legend()
plt.grid(True, alpha=0.3)

# Add performance benchmarks
for threshold in [0.3, 0.5, 0.7]:
    idx_rf = np.argmax(rf_results['y_pred_proba'] >= threshold)
    if idx_rf < len(fpr_rf):
        plt.scatter(fpr_rf[idx_rf], tpr_rf[idx_rf], s=100, 
                   label=f'RF threshold {threshold}', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate Precision-Recall curves
precision_rf, recall_rf, _ = precision_recall_curve(y_test, rf_results['y_pred_proba'])
precision_xgb, recall_xgb, _ = precision_recall_curve(y_test, xgb_results['y_pred_proba'])

# Calculate AUC-PR
from sklearn.metrics import auc
pr_auc_rf = auc(recall_rf, precision_rf)
pr_auc_xgb = auc(recall_xgb, precision_xgb)

plt.figure(figsize=(10, 8))
plt.plot(recall_rf, precision_rf, label=f'Random Forest (AUC-PR = {pr_auc_rf:.3f})', linewidth=2)
plt.plot(recall_xgb, precision_xgb, label=f'XGBoost (AUC-PR = {pr_auc_xgb:.3f})', linewidth=2)

# Add no-skill line (proportion of positive class)
no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [no_skill, no_skill], 'k--', label='No Skill')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Churn Prediction Models')
plt.legend()
plt.grid(True, alpha=0.3)

# Add threshold annotations
thresholds = [0.3, 0.5, 0.7]
for threshold in thresholds:
    idx_rf = np.argmax(rf_results['y_pred_proba'] >= threshold)
    if idx_rf < len(recall_rf):
        plt.scatter(recall_rf[idx_rf], precision_rf[idx_rf], s=100,
                   label=f'RF threshold {threshold}', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
def plot_confusion_matrix_heatmap(y_true, y_pred, model_name, ax):
    """Plot confusion matrix as heatmap"""
    cm = confusion_matrix(y_true, y_pred)
    cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    sns.heatmap(cm_percentage, annot=True, fmt='.1f', cmap='Blues', 
                xticklabels=['Predicted Not Churn', 'Predicted Churn'],
                yticklabels=['Actual Not Churn', 'Actual Churn'],
                cbar_kws={'label': 'Percentage'}, ax=ax)
    
    ax.set_title(f'{model_name}\nConfusion Matrix (%)')
    ax.set_ylabel('Actual')
    ax.set_xlabel('Predicted')

# Create subplots for both models
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

plot_confusion_matrix_heatmap(y_test, rf_results['y_pred'], 'Random Forest', ax1)
plot_confusion_matrix_heatmap(y_test, xgb_results['y_pred'], 'XGBoost', ax2)

plt.tight_layout()
plt.show()

# Print detailed confusion matrix analysis
def analyze_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{model_name} Confusion Matrix Analysis:")
    print(f"True Negatives:  {tn} (Correctly predicted not to churn)")
    print(f"False Positives: {fp} (Predicted to churn but didn't)")
    print(f"False Negatives: {fn} (Predicted not to churn but did)")
    print(f"True Positives:  {tp} (Correctly predicted to churn)")
    print(f"False Positive Rate: {fp/(fp+tn):.3f}")
    print(f"False Negative Rate: {fn/(fn+tp):.3f}")

analyze_confusion_matrix(y_test, rf_results['y_pred'], "Random Forest")
analyze_confusion_matrix(y_test, xgb_results['y_pred'], "XGBoost")

In [ ]:
# Random Forest Feature Importance
rf_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# XGBoost Feature Importance
xgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot feature importance comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Random Forest
ax1.barh(rf_importance['feature'].head(15), rf_importance['importance'].head(15))
ax1.set_title('Random Forest - Top 15 Feature Importance')
ax1.set_xlabel('Importance Score')

# XGBoost
ax2.barh(xgb_importance['feature'].head(15), xgb_importance['importance'].head(15))
ax2.set_title('XGBoost - Top 15 Feature Importance')
ax2.set_xlabel('Importance Score')

plt.tight_layout()
plt.show()

# Print top features
print("Top 10 Features - Random Forest:")
print(rf_importance.head(10).round(4))
print("\nTop 10 Features - XGBoost:")
print(xgb_importance.head(10).round(4))

In [ ]:
# Calibration curves
prob_true_rf, prob_pred_rf = calibration_curve(y_test, rf_results['y_pred_proba'], n_bins=10)
prob_true_xgb, prob_pred_xgb = calibration_curve(y_test, xgb_results['y_pred_proba'], n_bins=10)

plt.figure(figsize=(10, 8))
plt.plot(prob_pred_rf, prob_true_rf, 's-', label='Random Forest', linewidth=2)
plt.plot(prob_pred_xgb, prob_true_xgb, 's-', label='XGBoost', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Perfectly Calibrated')

plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Probability Calibration Curve')
plt.legend()
plt.grid(True, alpha=0.3)

# Add calibration metrics
from sklearn.calibration import CalibratedClassifierCV

# Calibrate the better performing model
calibrator = CalibratedClassifierCV(rf_model, method='isotonic', cv=5)
calibrator.fit(X_train, y_train)
y_calibrated_proba = calibrator.predict_proba(X_test)[:, 1]

prob_true_cal, prob_pred_cal = calibration_curve(y_test, y_calibrated_proba, n_bins=10)
plt.plot(prob_pred_cal, prob_true_cal, 's-', label='Calibrated Random Forest', linewidth=2)

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Select the best model based on business objectives
best_model = rf_model  # Based on our comparison
best_model_name = "Random Forest"

print(f"SELECTED MODEL: {best_model_name}")
print("="*50)

# Final performance summary
final_metrics = rf_results['metrics']
print(f"Final Model Performance:")
for metric, value in final_metrics.items():
    print(f"{metric.capitalize()}: {value:.4f}")

# Feature importance for business interpretation
top_features = rf_importance.head(10)
print(f"\nTop 10 Predictive Features:")
for i, (_, row) in enumerate(top_features.iterrows(), 1):
    print(f"{i}. {row['feature']}: {row['importance']:.4f}")

# Save model for deployment
import joblib
joblib.dump(best_model, 'churn_prediction_model.pkl')
joblib.dump(X.columns, 'feature_columns.pkl')

print("\nModel saved for deployment!")

****

# **Conclusion**
## Top 4 Critical Findings:
1. ### Contract Terms Dominate Churn Risk

* 42.7% of month-to-month customers churn vs only 2.9% of two-year contracts

* 15x higher churn risk for short-term contracts

* Priority Action: Convert month-to-month customers to longer commitments

2. ### New Customers Are Highest Risk

* 55.5% of churned customers leave within first year

* Churned customers have 40.1% lower total charges (newer relationships)

* Priority Action: Intensive onboarding and first-year retention programs

3. ### Service Quality Over Entertainment

* Security/Support services reduce churn by 63-65%

* Entertainment services only reduce churn by ~10%

* Priority Action: Bundle and promote security/support services aggressively

4. ### Payment & Billing Behavior Matters

* Electronic check users churn at 45.3% vs 15.2% for credit card users

* Paperless billing correlates with 33.6% churn rate

* Priority Action: Incentivize automatic payment methods

## Customer Risk Profile:
### 🚨 HIGHEST RISK CUSTOMER:
New customer with month-to-month contract, fiber optic internet, no security services, paying by electronic check

### ✅ LOWEST RISK CUSTOMER:
Long-term customer with two-year contract, multiple security services, paying by credit card



## Ahmed A. Elatwy

****